# Regression Predict Student Solution

© Explore Data Science Academy

---
### Honour Code

I {**HUZAIFA, ABU**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Predict Overview: Spain Electricity Shortfall Challenge

The government of Spain is considering an expansion of it's renewable energy resource infrastructure investments. As such, they require information on the trends and patterns of the countries renewable sources and fossil fuel energy generation. Your company has been awarded the contract to:

- 1. analyse the supplied data;
- 2. identify potential errors in the data and clean the existing data set;
- 3. determine if additional features can be added to enrich the data set;
- 4. build a model that is capable of forecasting the three hourly demand shortfalls;
- 5. evaluate the accuracy of the best machine learning model;
- 6. determine what features were most important in the model’s prediction decision, and
- 7. explain the inner working of the model to a non-technical audience.

Formally the problem statement was given to you, the senior data scientist, by your manager via email reads as follow:

> In this project you are tasked to model the shortfall between the energy generated by means of fossil fuels and various renewable sources - for the country of Spain. The daily shortfall, which will be referred to as the target variable, will be modelled as a function of various city-specific weather features such as `pressure`, `wind speed`, `humidity`, etc. As with all data science projects, the provided features are rarely adequate predictors of the target variable. As such, you are required to perform feature engineering to ensure that you will be able to accurately model Spain's three hourly shortfalls.
 
On top of this, she has provided you with a starter notebook containing vague explanations of what the main outcomes are. 

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Model Explanations</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Importing Packages ⚡ |
| :--------------------------- |
| In this section you are required to import, and briefly discuss, the libraries that will be used throughout your analysis and modelling. |

---

In [25]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np
import pandas as pd
pd.options.display.max_rows =100

# Libraries for data preparation and model building
import seaborn as sns
import matplotlib.pyplot as plt

# Setting global constants to ensure notebook results are reproducible
# #######PARAMETER_CONSTANT = ###

In [33]:
pd.options.display.max_rows =200
pd.options.display.max_columns =200

<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data ⚡ |
| :--------------------------- |
| In this section you are required to load the data from the `df_train` file into a DataFrame. |

---

In [4]:
raw_train = pd.read_csv('df_train.csv')# load the data
raw_test = pd.read_csv('df_test.csv')

In [6]:
raw_train.head()

,Unnamed: 0,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,...,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h
0,0,2015-01-01 03:00:00,0.666667,level_5,0.0,0.666667,74.333333,64.000000,0.0,1.000000,...,265.938000,281.013000,269.338615,269.338615,281.013000,269.338615,274.254667,265.938000,265.938000,6715.666667
1,1,2015-01-01 06:00:00,0.333333,level_10,0.0,1.666667,78.333333,64.666667,0.0,1.000000,...,266.386667,280.561667,270.376000,270.376000,280.561667,270.376000,274.945000,266.386667,266.386667,4171.666667
2,2,2015-01-01 09:00:00,1.000000,level_9,0.0,1.000000,71.333333,64.333333,0.0,1.000000,...,272.708667,281.583667,275.027229,275.027229,281.583667,275.027229,278.792000,272.708667,272.708667,4274.666667
3,3,2015-01-01 12:00:00,1.000000,level_8,0.0,1.000000,65.333333,56.333333,0.0,1.000000,...,281.895219,283.434104,281.135063,281.135063,283.434104,281.135063,285.394000,281.895219,281.895219,5075.666667
4,4,2015-01-01 15:00:00,1.000000,level_7,0.0,1.000000,59.000000,57.000000,2.0,0.333333,...,280.678437,284.213167,282.252063,282.252063,284.213167,282.252063,285.513719,280.678437,280.678437,6620.666667


In [48]:
raw_test.tail()

,Unnamed: 0,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,Seville_clouds_all,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,Madrid_clouds_all,Seville_wind_speed,Barcelona_rain_1h,Seville_pressure,Seville_rain_1h,Bilbao_snow_3h,Barcelona_pressure,Seville_rain_3h,Madrid_rain_1h,Barcelona_rain_3h,Valencia_snow_3h,Madrid_weather_id,Barcelona_weather_id,Bilbao_pressure,Seville_weather_id,Valencia_pressure,Seville_temp_max,Madrid_pressure,Valencia_temp_max,Valencia_temp,Bilbao_weather_id,Seville_temp,Valencia_humidity,Valencia_temp_min,Barcelona_temp_max,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min
2915,11678,2018-12-31 09:00:00,0.333333,level_9,0.0,2.000000,81.666667,49.666667,80.000000,1.333333,0.0,70.000000,6.333333,320.000000,0.0,2.000000,0.0,sp24,0.0,0,1029.666667,0.0,0.0,0.0,0,800.0,800.0,1034.666667,800.0,NaN,280.816667,1031.666667,279.816667,279.323333,734.333333,280.210000,69.666667,278.816667,279.816667,281.483333,278.140000,270.816667,273.210000,276.483333,276.150000,279.816667,274.910000,271.150000
2916,11679,2018-12-31 12:00:00,0.333333,level_8,0.0,1.333333,61.000000,28.333333,56.666667,1.000000,0.0,36.666667,2.666667,293.333333,0.0,4.000000,0.0,sp24,0.0,0,1028.666667,0.0,0.0,0.0,0,800.0,800.0,1034.333333,800.0,NaN,287.483333,1031.333333,290.816667,290.323333,747.666667,287.280000,32.000000,289.816667,286.483333,287.816667,286.150000,278.150000,278.443333,285.816667,278.816667,287.150000,283.156667,280.483333
2917,11680,2018-12-31 15:00:00,1.000000,level_6,0.0,3.000000,47.000000,26.333333,0.000000,0.666667,0.0,14.666667,2.666667,233.333333,0.0,4.666667,0.0,sp23,0.0,0,1027.000000,0.0,0.0,0.0,0,800.0,800.0,1033.000000,800.0,NaN,290.816667,1030.000000,293.483333,293.483333,800.000000,290.816667,23.666667,293.483333,289.483333,288.816667,288.820000,284.150000,285.073333,288.150000,285.816667,290.816667,287.733333,286.483333
2918,11681,2018-12-31 18:00:00,1.000000,level_6,0.0,2.000000,52.666667,56.666667,0.000000,0.666667,0.0,19.000000,2.333333,253.333333,0.0,3.333333,0.0,sp23,0.0,0,1027.000000,0.0,0.0,0.0,0,800.0,800.0,1033.333333,800.0,NaN,288.816667,1030.333333,286.483333,285.976667,800.000000,288.300000,59.000000,285.483333,285.816667,285.150000,284.473333,280.150000,281.626667,283.150000,282.816667,287.483333,283.813333,282.150000
2919,11682,2018-12-31 21:00:00,1.333333,level_10,0.0,2.333333,61.666667,69.333333,0.000000,1.333333,0.0,120.000000,4.666667,310.000000,0.0,3.666667,0.0,sp23,0.0,0,1028.000000,0.0,0.0,0.0,0,800.0,800.0,1034.000000,800.0,NaN,284.816667,1030.666667,281.150000,280.313333,800.000000,284.423333,79.000000,279.483333,283.816667,276.816667,281.133333,276.150000,276.450000,278.483333,276.816667,283.816667,276.623333,276.483333


<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Exploratory data analysis ⚡ |
| :--------------------------- |
| In this section, you are required to perform an in-depth analysis of all the variables in the DataFrame. |

---


looking at data statistics

In [47]:
raw_train.info().sort_values(value='dtype')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            8763 non-null   int64  
 1   time                  8763 non-null   object 
 2   Madrid_wind_speed     8763 non-null   float64
 3   Valencia_wind_deg     8763 non-null   object 
 4   Bilbao_rain_1h        8763 non-null   float64
 5   Valencia_wind_speed   8763 non-null   float64
 6   Seville_humidity      8763 non-null   float64
 7   Madrid_humidity       8763 non-null   float64
 8   Bilbao_clouds_all     8763 non-null   float64
 9   Bilbao_wind_speed     8763 non-null   float64
 10  Seville_clouds_all    8763 non-null   float64
 11  Bilbao_wind_deg       8763 non-null   float64
 12  Barcelona_wind_speed  8763 non-null   float64
 13  Barcelona_wind_deg    8763 non-null   float64
 14  Madrid_clouds_all     8763 non-null   float64
 15  Seville_wind_speed   

AttributeError: 'NoneType' object has no attribute 'sort_values'

In [21]:
#number of rows and columns in the dataset
n_rows_train = raw_train.shape[0]
n_cols_train = raw_train.shape[1]
print('There are {} rows  and {} columns in the dataset'.format(n_rows_train,n_cols_train)) 

There are 8763 rows  and 49 columns in the dataset


In [29]:
(raw_train.isnull().sum()/raw_train.shape[0]*100).sort_values(ascending=False)

Valencia_pressure       23.599224
Unnamed: 0               0.000000
Madrid_weather_id        0.000000
Bilbao_pressure          0.000000
Seville_weather_id       0.000000
Seville_temp_max         0.000000
Madrid_pressure          0.000000
Valencia_temp_max        0.000000
Valencia_temp            0.000000
Bilbao_weather_id        0.000000
Seville_temp             0.000000
Valencia_humidity        0.000000
Valencia_temp_min        0.000000
Barcelona_temp_max       0.000000
Madrid_temp_max          0.000000
Barcelona_temp           0.000000
Bilbao_temp_min          0.000000
Bilbao_temp              0.000000
Barcelona_temp_min       0.000000
Bilbao_temp_max          0.000000
Seville_temp_min         0.000000
Madrid_temp              0.000000
Madrid_temp_min          0.000000
Barcelona_weather_id     0.000000
Valencia_snow_3h         0.000000
time                     0.000000
Barcelona_rain_3h        0.000000
Madrid_wind_speed        0.000000
Valencia_wind_deg        0.000000
Bilbao_rain_1h

In [38]:
#number of unique values in each column
for x in raw_train:
    print('{} : {}'.format(x,raw_train[x].nunique()))

Unnamed: 0 : 8763
time : 8763
Madrid_wind_speed : 38
Valencia_wind_deg : 10
Bilbao_rain_1h : 20
Valencia_wind_speed : 58
Seville_humidity : 274
Madrid_humidity : 280
Bilbao_clouds_all : 277
Bilbao_wind_speed : 39
Seville_clouds_all : 246
Bilbao_wind_deg : 1042
Barcelona_wind_speed : 39
Barcelona_wind_deg : 974
Madrid_clouds_all : 250
Seville_wind_speed : 38
Barcelona_rain_1h : 31
Seville_pressure : 25
Seville_rain_1h : 17
Bilbao_snow_3h : 83
Barcelona_pressure : 189
Seville_rain_3h : 50
Madrid_rain_1h : 19
Barcelona_rain_3h : 77
Valencia_snow_3h : 6
Madrid_weather_id : 289
Barcelona_weather_id : 269
Bilbao_pressure : 205
Seville_weather_id : 301
Valencia_pressure : 143
Seville_temp_max : 3294
Madrid_pressure : 298
Valencia_temp_max : 3387
Valencia_temp : 4531
Bilbao_weather_id : 356
Seville_temp : 6702
Valencia_humidity : 260
Valencia_temp_min : 3458
Barcelona_temp_max : 2648
Madrid_temp_max : 3606
Barcelona_temp : 4233
Bilbao_temp_min : 3542
Bilbao_temp : 6676
Barcelona_temp_min : 257

In [73]:
#numeric columns and categorical columns
numeric_cols = raw_train.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_cols = raw_train.select_dtypes(include='object').columns.tolist()

#removing the unnamed column and the shortfall column
numeric_cols = numeric_cols[1:-1]
numeric_cols.sort()

In [82]:
print(numeric_cols)

['Barcelona_pressure', 'Barcelona_rain_1h', 'Barcelona_rain_3h', 'Barcelona_temp', 'Barcelona_temp_max', 'Barcelona_temp_min', 'Barcelona_weather_id', 'Barcelona_wind_deg', 'Barcelona_wind_speed', 'Bilbao_clouds_all', 'Bilbao_pressure', 'Bilbao_rain_1h', 'Bilbao_snow_3h', 'Bilbao_temp', 'Bilbao_temp_max', 'Bilbao_temp_min', 'Bilbao_weather_id', 'Bilbao_wind_deg', 'Bilbao_wind_speed', 'Madrid_clouds_all', 'Madrid_humidity', 'Madrid_pressure', 'Madrid_rain_1h', 'Madrid_temp', 'Madrid_temp_max', 'Madrid_temp_min', 'Madrid_weather_id', 'Madrid_wind_speed', 'Seville_clouds_all', 'Seville_humidity', 'Seville_rain_1h', 'Seville_rain_3h', 'Seville_temp', 'Seville_temp_max', 'Seville_temp_min', 'Seville_weather_id', 'Seville_wind_speed', 'Valencia_humidity', 'Valencia_pressure', 'Valencia_snow_3h', 'Valencia_temp', 'Valencia_temp_max', 'Valencia_temp_min', 'Valencia_wind_speed']


In [83]:
print(categorical_cols)

['time', 'Valencia_wind_deg', 'Seville_pressure']


In [ ]:
# plot relevant feature interactions

sns.

In [ ]:
# evaluate correlation

In [ ]:
# have a look at feature distributions

<a id="four"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |
| In this section you are required to: clean the dataset, and possibly create new features - as identified in the EDA phase. |

---

In [ ]:
# remove missing values/ features

In [ ]:
# create new features

In [ ]:
# engineer existing features

<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, you are required to create one or more regression models that are able to accurately predict the thee hour load shortfall. |

---

In [ ]:
# split data

In [ ]:
# create targets and features dataset

In [ ]:
# create one or more ML models

In [ ]:
# evaluate one or more ML models

<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section you are required to compare the relative performance of the various trained ML models on a holdout dataset and comment on what model is the best and why. |

---

In [ ]:
# Compare model performance

In [ ]:
# Choose best model and motivate why it is the best choice

<a id="seven"></a>
## 7. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, you are required to discuss how the best performing model works in a simple way so that both technical and non-technical stakeholders can grasp the intuition behind the model's inner workings. |

---

In [ ]:
# discuss chosen methods logic